In [232]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn import svm
import xgboost as xgb
from sklearn.metrics import r2_score
from sklearn import linear_model
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_hist_gradient_boosting  
from sklearn.ensemble import HistGradientBoostingRegressor

In [233]:
df_train=pd.read_csv("/train.csv")
df_song=pd.read_csv("/songs.csv")
df_song_label=pd.read_csv("/song_labels.csv")
df_save_for_later=pd.read_csv("/save_for_later.csv")
df_test=pd.read_csv("/test.csv")

In [234]:
index=df_test.index


In [235]:
df_train.describe()


,song_id,score
count,710109.000000,710109.000000
mean,2100.296939,3.935109
std,2536.273421,0.992082
min,1.000000,1.000000
25%,207.000000,3.000000
50%,944.000000,4.000000
75%,3182.000000,5.000000
max,10000.000000,5.000000


In [236]:
df_train.head(10)

,customer_id,song_id,score
0,O29219,3459,3
1,I50343,5326,4
2,N42888,236,5
3,F5740,724,4
4,K4115,8452,5
5,N19138,230,4
6,O7109,1247,2
7,J23614,1119,3
8,G29081,742,3
9,F47140,497,5


In [237]:
df_train["customer_id"].value_counts()

J15604    98
I24143    98
F6630     98
N9668     98
N10288    97
          ..
I40753     6
L25856     6
L42736     6
H13782     6
F34590     5
Name: customer_id, Length: 14053, dtype: int64

In [238]:
df_test["customer_id"].value_counts()

F6630     99
O10509    98
I24143    98
N9668     98
J15604    98
          ..
K51725    15
N32128    15
I40753    15
K43675    14
F34590    14
Name: customer_id, Length: 14053, dtype: int64

In [239]:
df_train["song_id"].value_counts()

1       2863
2       2657
4       2226
3       2175
17      1982
        ... 
9345       4
7803       4
9036       4
8520       4
9486       1
Name: song_id, Length: 10000, dtype: int64

In [240]:
df_train

,customer_id,song_id,score
0,O29219,3459,3
1,I50343,5326,4
2,N42888,236,5
3,F5740,724,4
4,K4115,8452,5
...,...,...,...
710104,H6322,1003,4
710105,L6306,705,5
710106,J2494,6028,3
710107,G41961,2397,4


In [241]:
df_train_final=pd.merge(left=df_train,right=df_song,on="song_id",how="left")
df_test_final=pd.merge(left=df_test,right=df_song,on="song_id",how="left")

In [242]:
df_train_final

,customer_id,song_id,score,platform_id,released_year,language,number_of_comments
0,O29219,3459,3,P49540,1782.0,eng,1066.0
1,I50343,5326,4,W216377,1981.0,eng,1119.0
2,N42888,236,5,X1898,1997.0,en-US,10439.0
3,F5740,724,4,W2233407,2008.0,eng,3500.0
4,K4115,8452,5,V5946,1987.0,NaN,762.0
...,...,...,...,...,...,...,...
710104,H6322,1003,4,V61666,1985.0,eng,2126.0
710105,L6306,705,5,U10605,1984.0,en-GB,1641.0
710106,J2494,6028,3,Y9509289,2011.0,en-US,2542.0
710107,G41961,2397,4,W76237,1985.0,NaN,36.0


In [243]:
df_song_label

,platform_id,label_id,count
0,Q207781,10304,6
1,S961833,29011,15
2,X6718608,25148,29
3,Q16134551,8717,774
4,V7126,15169,120
...,...,...,...
999907,P9530,31264,4
999908,V280366,15221,156
999909,R25052,11557,37
999910,R45432,30521,12


In [244]:
df_song_label.drop_duplicates(keep="first",inplace=True)
df_song_label.head()

,platform_id,label_id,count
0,Q207781,10304,6
1,S961833,29011,15
2,X6718608,25148,29
3,Q16134551,8717,774
4,V7126,15169,120


In [245]:
df_song_label.sort_values(by=["platform_id","count"],inplace=True)
df_song_label

,platform_id,label_id,count
43186,P10,5751,3
56700,P10,2218,3
57383,P10,14064,3
80747,P10,6909,3
94166,P10,33116,3
...,...,...,...
427784,Y9975779,20939,90
680283,Y9975779,8883,164
668638,Y9975779,11743,178
534203,Y9975779,8717,374


In [246]:
df_song_label.drop_duplicates("platform_id",keep="last",inplace=True)
df_song_label.head()

,platform_id,label_id,count
522972,P10,30574,3226
502330,P10090,30574,3005
42661,P102030,30574,11049
382884,P10210,30574,6084
954569,P1025250,30574,6583


In [247]:
df_train_final=pd.merge(left=df_train_final,right=df_song_label,on="platform_id",how="left")
df_test_final=pd.merge(left=df_test_final,right=df_song_label,on="platform_id",how="left")

In [248]:
df_train_final.head(5)

,customer_id,song_id,score,platform_id,released_year,language,number_of_comments,label_id,count
0,O29219,3459,3,P49540,1782.0,eng,1066.0,30574.0,31952.0
1,I50343,5326,4,W216377,1981.0,eng,1119.0,30574.0,14338.0
2,N42888,236,5,X1898,1997.0,en-US,10439.0,30574.0,21613.0
3,F5740,724,4,W2233407,2008.0,eng,3500.0,30574.0,22397.0
4,K4115,8452,5,V5946,1987.0,NaN,762.0,30574.0,5715.0


In [249]:
df_save_for_later

,customer_id,song_id
0,F22180,1140
1,N34498,1523
2,K46875,6918
3,L50176,5828
4,F13830,3039
...,...,...
238875,J52814,5662
238876,G35381,1216
238877,L716,4581
238878,I15343,16


In [250]:
prefered=[]
for i in range(238880):
    prefered.append(1)
df_save_for_later["prefered"]=prefered

In [251]:
df_train_final=pd.merge(left=df_train_final,right=df_save_for_later,on=["song_id","customer_id"],how="left")
df_test_final=pd.merge(left=df_test_final,right=df_save_for_later,on=["song_id","customer_id"],how="left")

In [252]:
df_test_final

,customer_id,song_id,platform_id,released_year,language,number_of_comments,label_id,count,prefered
0,J24644,5304,U64955,1989.0,NaN,416.0,30574.0,2386.0,NaN
1,G29701,123,X5358,1991.0,eng,3139.0,30574.0,2988.0,NaN
2,M29977,11,S77203,2003.0,eng,59730.0,11557.0,12736.0,NaN
3,H50752,1445,P331920,2001.0,en-US,6209.0,30574.0,35663.0,NaN
4,J42394,271,S18373,1966.0,en-US,11328.0,30574.0,17720.0,NaN
...,...,...,...,...,...,...,...,...,...
670676,J20614,92,R1202,2005.0,en-US,13668.0,30574.0,9160.0,NaN
670677,F11740,21,R2,2003.0,eng,28685.0,30574.0,24549.0,NaN
670678,I43093,2294,W7137327,2011.0,eng,4692.0,30574.0,91947.0,NaN
670679,J26924,1216,S3273,2003.0,en-US,7740.0,30574.0,70720.0,NaN


In [253]:
df_train_final["prefered"].fillna(0,inplace=True)
df_test_final["prefered"].fillna(0,inplace=True)

In [254]:
df_test_final.head(10)

,customer_id,song_id,platform_id,released_year,language,number_of_comments,label_id,count,prefered
0,J24644,5304,U64955,1989.0,NaN,416.0,30574.0,2386.0,0.0
1,G29701,123,X5358,1991.0,eng,3139.0,30574.0,2988.0,0.0
2,M29977,11,S77203,2003.0,eng,59730.0,11557.0,12736.0,0.0
3,H50752,1445,P331920,2001.0,en-US,6209.0,30574.0,35663.0,0.0
4,J42394,271,S18373,1966.0,en-US,11328.0,30574.0,17720.0,0.0
5,L51186,36,V3636,1993.0,eng,54084.0,30574.0,8433.0,0.0
6,L4766,8,W5107,1951.0,eng,44920.0,7457.0,22118.0,0.0
7,N26388,6344,Q2210921,2008.0,eng,1055.0,30574.0,5660.0,0.0
8,L986,245,Y2839,1977.0,eng,9270.0,30574.0,15173.0,0.0
9,K40485,400,X22328,1984.0,eng,6623.0,30574.0,23015.0,0.0


In [255]:
df_test_final["language"].bfill(inplace=True)
df_test_final.head()

,customer_id,song_id,platform_id,released_year,language,number_of_comments,label_id,count,prefered
0,J24644,5304,U64955,1989.0,eng,416.0,30574.0,2386.0,0.0
1,G29701,123,X5358,1991.0,eng,3139.0,30574.0,2988.0,0.0
2,M29977,11,S77203,2003.0,eng,59730.0,11557.0,12736.0,0.0
3,H50752,1445,P331920,2001.0,en-US,6209.0,30574.0,35663.0,0.0
4,J42394,271,S18373,1966.0,en-US,11328.0,30574.0,17720.0,0.0


In [256]:
df_train_final.dropna(inplace=True)

In [257]:
df_train_final["language"].value_counts()

eng      501948
en-US    138214
en-GB     13421
en-CA      5568
ara        2921
spa        2898
fre        1255
ind         708
ger         486
jpn         197
por         184
en          170
per         166
nl          104
nor         102
pol          84
dan          64
ita          40
vie          31
mul          23
swe          15
tur          14
rus          13
fil          13
rum          12
Name: language, dtype: int64

In [258]:
le = preprocessing.LabelEncoder()

df_train_final['customer_id']= le.fit_transform(df_train_final['customer_id'])
df_train_final['platform_id']= le.fit_transform(df_train_final['platform_id'])
df_test_final['customer_id']= le.fit_transform(df_test_final['customer_id'])


In [259]:
df_test_final["released_year"].value_counts()

2012.0    34174
2011.0    30621
2013.0    26991
2005.0    26063
2009.0    25221
          ...  
1532.0       13
119.0        12
1418.0       11
1590.0       10
1825.0        5
Name: released_year, Length: 292, dtype: int64

In [260]:
df_train_final.head()

,customer_id,song_id,score,platform_id,released_year,language,number_of_comments,label_id,count,prefered
0,13168,3459,3,624,1782.0,eng,1066.0,30574.0,31952.0,0.0
1,5393,5326,4,6573,1981.0,eng,1119.0,30574.0,14338.0,0.0
2,12189,236,5,7421,1997.0,en-US,10439.0,30574.0,21613.0,0.0
3,1306,724,4,6588,2008.0,eng,3500.0,30574.0,22397.0,0.0
5,11498,230,4,5695,1817.0,eng,13117.0,30574.0,220502.0,0.0


In [261]:
indexNames = df_train_final[ df_train_final['released_year'] < 1796].index
df_train_final.drop(indexNames , inplace=True)



In [262]:
df_language=pd.get_dummies(df_train_final['language'], prefix='language')
df_train_final= pd.concat([df_train_final, df_language],axis=1)

df_language_test=pd.get_dummies(df_test_final['language'], prefix='language')
df_test_final= pd.concat([df_test_final, df_language_test],axis=1)

In [263]:
df_test_final.head()

,customer_id,song_id,platform_id,released_year,language,number_of_comments,label_id,count,prefered,language_ara,language_dan,language_en,language_en-CA,language_en-GB,language_en-US,language_eng,language_fil,language_fre,language_ger,language_ind,language_ita,language_jpn,language_mul,language_nl,language_nor,language_per,language_pol,language_por,language_rum,language_rus,language_spa,language_swe,language_tur,language_vie
0,6029,5304,U64955,1989.0,eng,416.0,30574.0,2386.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1938,123,X5358,1991.0,eng,3139.0,30574.0,2988.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,10396,11,S77203,2003.0,eng,59730.0,11557.0,12736.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3980,1445,P331920,2001.0,en-US,6209.0,30574.0,35663.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,6575,271,S18373,1966.0,en-US,11328.0,30574.0,17720.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [264]:
df_train_final.head()

,customer_id,song_id,score,platform_id,released_year,language,number_of_comments,label_id,count,prefered,language_ara,language_dan,language_en,language_en-CA,language_en-GB,language_en-US,language_eng,language_fil,language_fre,language_ger,language_ind,language_ita,language_jpn,language_mul,language_nl,language_nor,language_per,language_pol,language_por,language_rum,language_rus,language_spa,language_swe,language_tur,language_vie
1,5393,5326,4,6573,1981.0,eng,1119.0,30574.0,14338.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,12189,236,5,7421,1997.0,en-US,10439.0,30574.0,21613.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1306,724,4,6588,2008.0,eng,3500.0,30574.0,22397.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,11498,230,4,5695,1817.0,eng,13117.0,30574.0,220502.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,13976,1247,2,5720,1967.0,eng,2256.0,30574.0,2448.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [265]:
df_train_final.drop("platform_id",axis=1,inplace=True)
df_test_final.drop("platform_id",axis=1,inplace=True)

In [266]:
df_train_final.drop("language",axis=1,inplace=True)
df_test_final.drop("language",axis=1,inplace=True)

df_train_final.head()

,customer_id,song_id,score,released_year,number_of_comments,label_id,count,prefered,language_ara,language_dan,language_en,language_en-CA,language_en-GB,language_en-US,language_eng,language_fil,language_fre,language_ger,language_ind,language_ita,language_jpn,language_mul,language_nl,language_nor,language_per,language_pol,language_por,language_rum,language_rus,language_spa,language_swe,language_tur,language_vie
1,5393,5326,4,1981.0,1119.0,30574.0,14338.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,12189,236,5,1997.0,10439.0,30574.0,21613.0,0.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1306,724,4,2008.0,3500.0,30574.0,22397.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,11498,230,4,1817.0,13117.0,30574.0,220502.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,13976,1247,2,1967.0,2256.0,30574.0,2448.0,0.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [267]:
df_train_final["customer_id"] /df_train_final["customer_id"].abs().max()
df_train_final["song_id"] /df_train_final["song_id"].abs().max()
df_train_final["released_year"] /df_train_final["released_year"].abs().max()
df_train_final["label_id"] /df_train_final["label_id"].abs().max()
df_train_final["count"] /df_train_final["count"].abs().max()
df_train_final["number_of_comments"] /df_train_final["number_of_comments"].abs().max()

df_test_final["customer_id"] /df_test_final["customer_id"].abs().max()
df_test_final["song_id"] /df_test_final["song_id"].abs().max()
df_test_final["released_year"] /df_test_final["released_year"].abs().max()
df_test_final["label_id"] /df_test_final["label_id"].abs().max()
df_test_final["count"] /df_test_final["count"].abs().max()
df_test_final["number_of_comments"] /df_test_final["number_of_comments"].abs().max()

0         0.002679
1         0.020218
2         0.384724
3         0.039993
4         0.072964
            ...   
670676    0.088036
670677    0.184762
670678    0.030221
670679    0.049854
670680    0.007935
Name: number_of_comments, Length: 670681, dtype: float64

In [268]:
x_train,x_val,y_train,y_val= train_test_split(df_train_final.drop(["score"],axis=1),df_train_final["score"],random_state=17)

In [269]:
#parameters={'loss':['least_squares', 'least_absolute_deviation', 'poisson'],'learning_rate':[0.01, 0.05, 0.1, 0.5, 1],'max_iter':[50,100, 500, 700, 1000] , 'max_depth':[None, 10, 20, 50, 100],'l2_regularization':[0,0.01,0.1,0.2,0.5,1],'max_bins':[100 , 150 , 225 , 250],'warm_start':[True, False]}
#model =GridSearchCV(HistGradientBoostingRegressor(random_state=44),param_grid=parameters,cv=5

model=HistGradientBoostingClassifier(learning_rate=0.1,l2_regularization=0.01,max_iter=600,max_bins=255,max_depth=100,min_samples_leaf=50,warm_start=True,random_state=44)
model.fit(x_train,y_train)

In [270]:
model=HistGradientBoostingRegressor(learning_rate=0.17,l2_regularization=0,max_iter=800,max_bins=255,max_depth=100,min_samples_leaf=50,warm_start=True,random_state=44) 
model.fit(x_train,y_train)

HistGradientBoostingRegressor(l2_regularization=0, learning_rate=0.17,
                              loss='least_squares', max_bins=255, max_depth=100,
                              max_iter=800, max_leaf_nodes=31,
                              min_samples_leaf=50, n_iter_no_change=None,
                              random_state=44, scoring=None, tol=1e-07,
                              validation_fraction=0.1, verbose=0,
                              warm_start=True)

In [271]:
y_train_pred=model.predict(x_train)
y_val_pred=model.predict(x_val)

In [272]:
print("Training R2_score:",r2_score(y_train,y_train_pred))
print("Validation R2_score:",r2_score(y_val,y_val_pred))

Training R2_score: 0.11600222353004352
Validation R2_score: 0.06185489710902481


In [273]:
mse_train=mean_squared_error(y_train,y_train_pred)
mse_val=mean_squared_error(y_val,y_val_pred)

In [274]:
print("Training MSE:",mse_train)
print("Validation MSE:",mse_val)

Training MSE: 0.8703448487687436
Validation MSE: 0.9190554341756213


In [275]:
y_test_pred=model.predict(df_test_final)

In [276]:
y_test_pred

array([3.78524346, 3.90774892, 3.97073092, ..., 3.69649342, 3.97479838,
       3.63835332])

In [277]:
y_submission = {'test_row_id':index,
        'score':y_test_pred}
  

dftest_submission = pd.DataFrame(y_submission)
dftest_submission

,test_row_id,score
0,0,3.785243
1,1,3.907749
2,2,3.970731
3,3,3.958088
4,4,4.038717
...,...,...
670676,670676,3.636907
670677,670677,4.070894
670678,670678,3.696493
670679,670679,3.974798


In [278]:
score_round=dftest_submission["score"].round(decimals=1)

In [279]:
dftest_submission["score"]=score_round

In [280]:
dftest_submission


,test_row_id,score
0,0,3.8
1,1,3.9
2,2,4.0
3,3,4.0
4,4,4.0
...,...,...
670676,670676,3.6
670677,670677,4.1
670678,670678,3.7
670679,670679,4.0


In [281]:
dftest_submission.to_csv("/submission.csv",index=False)